In [1]:
import pandas as pd
import numpy as np

In [4]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)


In [5]:
# model1_result = "/home2/s439906/project/CASP16/oligomer/1_model_6/group_by_target_EU_new/dockq_ave-1-all-impute=-2_raw.csv"
# model6_result = "/home2/s439906/project/CASP16/oligomer/1_model_6/group_by_target_EU_new/dockq_ave-6-all-impute=-2_raw.csv"
# model1_result = "/home2/s439906/project/CASP16/oligomer/1_model_6/group_by_target_EU_new/qs_global-1-all-impute=-2_raw.csv"
# model6_result = "/home2/s439906/project/CASP16/oligomer/1_model_6/group_by_target_EU_new/qs_global-6-all-impute=-2_raw.csv"
model1_result = "/home2/s439906/project/CASP16/oligomer/group_by_target_EU_new/qs_global-first-all-impute=-2_raw.csv"
model6_result = "/home2/s439906/project/CASP16/oligomer/group_by_target_EU_new/qs_global-sixth-all-impute=-2_raw.csv"

model1 = pd.read_csv(model1_result, index_col=0)
model6 = pd.read_csv(model6_result, index_col=0)

In [ ]:
# get the comm rows in two dataframes
common_rows = model1.index.intersection(model6.index)
# get the df with common rows
model1_common = model1.loc[common_rows]
model6_common = model6.loc[common_rows]
# print(model1_common)
# print(model6_common)
model1_common

In [ ]:
model6_common

In [ ]:
# element-wise division of two dataframes
division = model6_common.div(model1_common)
mask = division.isna()
division

In [ ]:
# sum of the division
# fill division by 0
division_fill = division.fillna(0)
# only average on non-NaN values
# check if there are any inf values

# 假设 df 是你的 DataFrame
# 检查是否存在 inf 值
has_inf = np.isinf(division_fill.values).any()
print(has_inf)

# replace inf with 1
division_fill = division_fill.replace([np.inf, -np.inf], 10)
sum_division = division_fill.sum(axis=1)


In [ ]:

# divide the sum by the number of non-NaN values for each row
num_non_nan = division.count(axis=1)
num_non_nan

average_division = sum_division / num_non_nan
average_division

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
from matplotlib.colors import ListedColormap
import numpy as np

# sum = division_fill.sum(axis=1)
sum = average_division
sorted_indices = sum.sort_values(ascending=True).index
sorted_heatmap_data = division_fill.loc[sorted_indices].reset_index(
    drop=True)
sorted_sum = sum.loc[sorted_indices].reset_index(drop=True)
sorted_mask = pd.DataFrame(
    mask, index=division_fill.index).loc[sorted_indices].reset_index(drop=True)
# use mask to mask the data. will be used for heatmap
masked_data = sorted_heatmap_data.copy()
masked_data[sorted_mask] = np.nan
# set up the colormap
cmap = plt.cm.YlGn
cmap = ListedColormap(cmap(np.linspace(0, 1, 256)))
cmap.set_bad(color='gray')  # set the masked area to gray
# set up the figure and gridspec
fig = plt.figure(figsize=(32, 4), dpi=300)
gs = GridSpec(1, 2, width_ratios=[4, 1], wspace=0.04)
# plot the heatmap
ax0 = fig.add_subplot(gs[0])
sns.heatmap(masked_data, cmap=cmap, cbar=True, ax=ax0)
ax0.set_yticklabels(
    [f'{i}' for i in sorted_indices], rotation=0)  # use the same order as the row sum
ax0.set_xticklabels(sorted_heatmap_data.columns, rotation=90)
# set x tick font size
ax0.tick_params(axis='x', labelsize=16)
# set y tick font size
ax0.tick_params(axis='y', labelsize=16)
# set the font size of the colorbar
cbar = ax0.collections[0].colorbar
cbar.ax.tick_params(labelsize=16)

ax0.set_title(
    "model 6 (QS global) / model 1 (QS global)", fontsize=20)
# plot the row sum
ax1 = fig.add_subplot(gs[1], sharey=ax0)
y_pos = range(len(sorted_sum))
y_pos = [i+0.5 for i in y_pos]  # change the position of the bars
ax1.barh(y_pos, sorted_sum, color='tan')
# ax1.margins(y=0.5)
ax1.set_yticks(range(len(sorted_sum)))
ax1.set_yticklabels(
    [f'{i}' for i in sorted_indices], rotation=0)  # use the same order as the heatmap
# ax1.spines['bottom'].set_position(('outward', 10))  # 将 x 轴向下移动 10 点
# ymin, ymax = ax1.get_ylim()  # 获取当前的 y 轴范围
# ax1.set_ylim(ymin - 1, ymax-1)  # 为最底部条形预留空间
# set x tick font size
ax1.tick_params(axis='x', labelsize=16)
# set y tick font size
ax1.tick_params(axis='y', labelsize=16)
ax1.invert_yaxis()  # flip the y-axis
ax1.set_xlabel("Sum", fontsize=16)
ax1.set_title("Group average", fontsize=20)
# draw a line at x=1 
ax1.axvline(x=1, color='r', linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
from matplotlib.colors import ListedColormap
import numpy as np

# sum = division_fill.sum(axis=1)
sum = average_division
sum = sum.drop(['TS294',  'TS369'])
sorted_indices = sum.sort_values(ascending=True).index
sorted_heatmap_data = division_fill.loc[sorted_indices].reset_index(
    drop=True)
sorted_sum = sum.loc[sorted_indices].reset_index(drop=True)
sorted_mask = pd.DataFrame(
    mask, index=division_fill.index).loc[sorted_indices].reset_index(drop=True)
# use mask to mask the data. will be used for heatmap
masked_data = sorted_heatmap_data.copy()
masked_data[sorted_mask] = np.nan
# set up the colormap
cmap = plt.cm.YlGn
cmap = ListedColormap(cmap(np.linspace(0, 1, 256)))
cmap.set_bad(color='gray')  # set the masked area to gray
# set up the figure and gridspec
fig = plt.figure(figsize=(32, 4), dpi=300)
gs = GridSpec(1, 2, width_ratios=[4, 1], wspace=0.04)
# plot the heatmap
ax0 = fig.add_subplot(gs[0])
sns.heatmap(masked_data, cmap=cmap, cbar=True, ax=ax0)
ax0.set_yticklabels(
    [f'{i}' for i in sorted_indices], rotation=0)  # use the same order as the row sum
ax0.set_xticklabels(sorted_heatmap_data.columns, rotation=90)
# set x tick font size
ax0.tick_params(axis='x', labelsize=16)
# set y tick font size
ax0.tick_params(axis='y', labelsize=16)
# set the font size of the colorbar
cbar = ax0.collections[0].colorbar
cbar.ax.tick_params(labelsize=16)

ax0.set_title(
    "model 6 (QS global) / model 1 (QS global)", fontsize=20)
# plot the row sum
ax1 = fig.add_subplot(gs[1], sharey=ax0)
y_pos = range(len(sorted_sum))
y_pos = [i+0.5 for i in y_pos]  # change the position of the bars
ax1.barh(y_pos, sorted_sum, color='tan')
# ax1.margins(y=0.5)
ax1.set_yticks(range(len(sorted_sum)))
ax1.set_yticklabels(
    [f'{i}' for i in sorted_indices], rotation=0)  # use the same order as the heatmap
# ax1.spines['bottom'].set_position(('outward', 10))  # 将 x 轴向下移动 10 点
# ymin, ymax = ax1.get_ylim()  # 获取当前的 y 轴范围
# ax1.set_ylim(ymin - 1, ymax-1)  # 为最底部条形预留空间
# set x tick font size
ax1.tick_params(axis='x', labelsize=16)
# set y tick font size
ax1.tick_params(axis='y', labelsize=16)
ax1.invert_yaxis()  # flip the y-axis
ax1.set_xlabel("Sum", fontsize=16)
ax1.set_title("Group average", fontsize=20)
# draw a line at x=1 
ax1.axvline(x=1, color='r', linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
from matplotlib.colors import ListedColormap
import numpy as np

# sum = division_fill.sum(axis=1)
sum = average_division
sum = sum.drop(['TS294', 'TS139', 'TS059', 'TS369'])
sorted_indices = sum.sort_values(ascending=True).index
sorted_heatmap_data = division_fill.loc[sorted_indices].reset_index(
    drop=True)
sorted_sum = sum.loc[sorted_indices].reset_index(drop=True)
sorted_mask = pd.DataFrame(
    mask, index=division_fill.index).loc[sorted_indices].reset_index(drop=True)
# use mask to mask the data. will be used for heatmap
masked_data = sorted_heatmap_data.copy()
masked_data[sorted_mask] = np.nan
# set up the colormap
cmap = plt.cm.YlGn
cmap = ListedColormap(cmap(np.linspace(0, 1, 256)))
cmap.set_bad(color='gray')  # set the masked area to gray
# set up the figure and gridspec
fig = plt.figure(figsize=(32, 4), dpi=300)
gs = GridSpec(1, 2, width_ratios=[4, 1], wspace=0.04)
# plot the heatmap
ax0 = fig.add_subplot(gs[0])
sns.heatmap(masked_data, cmap=cmap, cbar=True, ax=ax0)
ax0.set_yticklabels(
    [f'{i}' for i in sorted_indices], rotation=0)  # use the same order as the row sum
ax0.set_xticklabels(sorted_heatmap_data.columns, rotation=90)
# set x tick font size
ax0.tick_params(axis='x', labelsize=16)
# set y tick font size
ax0.tick_params(axis='y', labelsize=16)
# set the font size of the colorbar
cbar = ax0.collections[0].colorbar
cbar.ax.tick_params(labelsize=16)

ax0.set_title(
    "model 6 (QS global) / model 1 (QS global)", fontsize=20)
# plot the row sum
ax1 = fig.add_subplot(gs[1], sharey=ax0)
y_pos = range(len(sorted_sum))
y_pos = [i+0.5 for i in y_pos]  # change the position of the bars
ax1.barh(y_pos, sorted_sum, color='tan')
# ax1.margins(y=0.5)
ax1.set_yticks(range(len(sorted_sum)))
ax1.set_yticklabels(
    [f'{i}' for i in sorted_indices], rotation=0)  # use the same order as the heatmap
# ax1.spines['bottom'].set_position(('outward', 10))  # 将 x 轴向下移动 10 点
# ymin, ymax = ax1.get_ylim()  # 获取当前的 y 轴范围
# ax1.set_ylim(ymin - 1, ymax-1)  # 为最底部条形预留空间
# set x tick font size
ax1.tick_params(axis='x', labelsize=16)
# set y tick font size
ax1.tick_params(axis='y', labelsize=16)
ax1.invert_yaxis()  # flip the y-axis
ax1.set_xlabel("Sum", fontsize=16)
ax1.set_title("Group average", fontsize=20)
# draw a line at x=1 
ax1.axvline(x=1, color='r', linestyle='--')
plt.tight_layout()
plt.show()